# Supporting Microsoft’s GraphRAG: Part 2

In the [previous section](msft_graphrag_1.md), we utilized Microsoft's GraphRAG to transform unstructured documents into Parquet files. Using TigerGraphX, we then converted these files into CSV format, created a graph in TigerGraph, and loaded the CSV data into it.

Now, let’s use Jupyter Notebook to explore the graph data and perform graph analysis.

To run this Jupyter Notebook, you can download the original `.ipynb` file from [msft_graphrag_2.ipynb](https://github.com/xuanleilin/tigergraphx/tree/main/docs/graphrag/msft_graphrag_2.ipynb).

---

## Get the Graph from TigerGraph
Since the graph has already been created in TigerGraph, redefining its schema is unnecessary. Instead, you can provide the graph name to retrieve it. TigerGraphX will verify if the graph exists in TigerGraph and, if it does, will return the corresponding graph.

In [2]:
from tigergraphx import Graph, TigerGraphConnectionConfig
connection = TigerGraphConnectionConfig.ensure_config({
    "host": "http://127.0.0.1",
    "user_name": "tigergraph",
    "password": "tigergraph",
})
G = Graph.from_db("GraphRAG", connection)

## Display the Graph Schema

Let's retrieve the graph schema using the `get_schema` method. The output is a Python dictionary containing three keys: `"graph_name"`, `"nodes"`, and `"edges"`. We'll print each of them individually to explore the schema details.
### Retrieve the Graph Schema and Display the Graph Name

In [3]:
schema = G.get_schema()
schema["graph_name"]

'GraphRAG'

### Display the Node Tyeps

In [4]:
for node in schema["nodes"].items():
    print(node)

('Document', {'primary_key': 'id', 'attributes': {'title': {'data_type': <DataType.STRING: 'STRING'>, 'default_value': None}, 'id': {'data_type': <DataType.STRING: 'STRING'>, 'default_value': None}}})
('TextUnit', {'primary_key': 'id', 'attributes': {'text': {'data_type': <DataType.STRING: 'STRING'>, 'default_value': None}, 'n_tokens': {'data_type': <DataType.UINT: 'UINT'>, 'default_value': None}, 'id': {'data_type': <DataType.STRING: 'STRING'>, 'default_value': None}}})
('Entity', {'primary_key': 'id', 'attributes': {'human_readable_id': {'data_type': <DataType.UINT: 'UINT'>, 'default_value': None}, 'name': {'data_type': <DataType.STRING: 'STRING'>, 'default_value': None}, 'entity_type': {'data_type': <DataType.STRING: 'STRING'>, 'default_value': None}, 'description': {'data_type': <DataType.STRING: 'STRING'>, 'default_value': None}, 'id': {'data_type': <DataType.STRING: 'STRING'>, 'default_value': None}}})
('Relationship', {'primary_key': 'id', 'attributes': {'human_readable_id': {'d

### Display the Edge Types

In [5]:
for edge in schema["edges"].items():
    print(edge)

('document_contains_text_unit', {'is_directed_edge': False, 'from_node_type': 'Document', 'to_node_type': 'TextUnit', 'attributes': {}})
('text_unit_contains_entity', {'is_directed_edge': False, 'from_node_type': 'TextUnit', 'to_node_type': 'Entity', 'attributes': {}})
('text_unit_contains_relationship', {'is_directed_edge': False, 'from_node_type': 'TextUnit', 'to_node_type': 'Relationship', 'attributes': {}})
('relationship_source', {'is_directed_edge': False, 'from_node_type': 'Relationship', 'to_node_type': 'Entity', 'attributes': {}})
('relationship_target', {'is_directed_edge': False, 'from_node_type': 'Relationship', 'to_node_type': 'Entity', 'attributes': {}})
('community_contains_entity', {'is_directed_edge': False, 'from_node_type': 'Community', 'to_node_type': 'Entity', 'attributes': {}})
('community_contains_relationship', {'is_directed_edge': False, 'from_node_type': 'Community', 'to_node_type': 'Relationship', 'attributes': {}})


## Display Node and Edge Counts

Gain deeper insights into the graph by exploring details such as the total number of nodes and the count of nodes for each node type.

### Display the Total Number of Nodes

In [6]:
G.number_of_nodes()

931

### Display the Count of Nodes for Each Node Type

In [7]:
for node_type in schema["nodes"]:
    print(f"{node_type}: {G.number_of_nodes(node_type)}")

Document: 1
TextUnit: 47
Entity: 398
Relationship: 426
Community: 59


### Display the Total Number of Edges

In [8]:
G.number_of_edges()

3630

### Display the Count of Edges for Each Edge Type

In [9]:
for edge_type in schema["edges"]:
    print(f"{edge_type}: {G.number_of_edges(edge_type)}")

document_contains_text_unit: 47
text_unit_contains_entity: 699
text_unit_contains_relationship: 517
relationship_source: 426
relationship_target: 426
community_contains_entity: 566
community_contains_relationship: 949


## Retrieve Sample Nodes from the Graph

In [10]:
G.get_nodes(node_type="Entity", limit=2)

,v_id,v_type,human_readable_id,entity_type,name,description,id
0,7373c84a439841d580b4650dac71136f,Entity,328,ORGANIZATION,HEMERUS CORPORATION,Hemerus Corporation is mentioned as one of the...,7373c84a439841d580b4650dac71136f
1,4a5ddbde3f354a79bb5ae3436ab67d25,Entity,294,ORGANIZATION,RENALGUARD,"RenalGuard, developed by RenalGuard Solutions,...",4a5ddbde3f354a79bb5ae3436ab67d25


In [11]:
G.get_nodes(node_type="Relationship", limit=2)

,v_id,v_type,human_readable_id,rank,weight,description,id
0,f09d47220cf146a4bc9869976740f230,Relationship,111,122,1,CytoSorb received funding from JPEO-CBD for a ...,f09d47220cf146a4bc9869976740f230
1,89d0f67d14504ef19601f6ff0eb2b32c,Relationship,7,126,35,"CytoSorb, a therapeutic device, has made signi...",89d0f67d14504ef19601f6ff0eb2b32c


In [12]:
G.get_nodes(node_type="Community", limit=2)

,v_id,v_type,summary,level,full_content,rank,id,rank_explanation,title
0,8,Community,This report focuses on Baxter and its signific...,0,# Baxter's Innovations in Dialysis and Sepsis ...,8.5,8,The high impact severity rating reflects Baxte...,Community 8
1,16,Community,"The REMOVE study, focusing on the use of CytoS...",0,# REMOVE Study and Its Oversight Community\n\n...,7.5,16,The impact severity rating is high due to the ...,Community 16


---

## What’s Next?

- [Supporting Microsoft’s GraphRAG: Part 3](../msft_graphrag_3): Perform queries using GSQL and Python-native TigerGraphX, with global and local context builders.

---

Start transforming your GraphRAG workflows with the power of **TigerGraphX** today! 